In [1]:
import pickle
import os
import numpy as np
import tensorflow as tf
import pandas as pd

In [2]:
X,y = pickle.load(open('../data/xyhighpass.p','rb'))

In [3]:
X = np.nan_to_num(X[:,:,:3])
y = np.array(y)
# groups = np.array(np.argmax(y,axis=1)).reshape(-1)
groups = y
# X = X - np.percentile(X,50)/(np.percentile(X,75)-np.percentile(X,25))

In [4]:
def separate_known_unknown_users(X,y,groups,split_ratio = .2):
    from sklearn.model_selection import LeavePGroupsOut
    n_users = len(np.unique(groups))
    lpgo = LeavePGroupsOut(n_groups=int(n_users*split_ratio))
    for train_index, test_index in lpgo.split(X, y, groups=groups):
        X_keep, X_out = X[train_index], X[test_index]
        y_keep, y_out = y[train_index], y[test_index]
        groups_keep, groups_out = groups[train_index], groups[test_index]
        break
    return X_keep,X_out,y_keep,y_out,groups_keep,groups_out

In [5]:
X_keep,X_test_out,y_keep,y_test_out,groups_keep,groups_test_out = separate_known_unknown_users(X,y,groups,split_ratio=.2)

X_in,X_val_out,y_in,y_val_out,groups_in,groups_val_out = separate_known_unknown_users(X_keep,y_keep,groups_keep,split_ratio=.1)

from sklearn.model_selection import train_test_split
X_train, X_test_in, y_train, y_test_in,groups_train,groups_test_in = train_test_split(X_in, y_in,groups_in, test_size=0.2, random_state=42,stratify=groups_in)

X_train_in, X_val_in, y_train_in, y_val_in,groups_train_in,groups_val_in = train_test_split(X_train, y_train,groups_train, test_size=0.1, random_state=42,stratify=groups_train)

In [9]:
# np.unique(groups_train_in),np.unique(groups_val_in),np.unique(groups_test_in)

In [10]:
train_data = pd.DataFrame({'x':list(X_train_in),'y':list(y_train_in),'group':groups_train_in})
val_data_in = pd.DataFrame({'x':list(X_val_in),'y':list(y_val_in),'group':groups_val_in})
val_data_out = pd.DataFrame({'x':list(X_val_out),'y':list(y_val_out),'group':groups_val_out})
test_data_in = pd.DataFrame({'x':list(X_test_in),'y':list(y_test_in),'group':groups_test_in})
test_data_out = pd.DataFrame({'x':list(X_test_out),'y':list(y_test_out),'group':groups_test_out})

In [11]:
from keras.layers import *
from keras.models import *
from keras import regularizers
from keras import backend as K
import matplotlib.pyplot as plt
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import numpy as np
%matplotlib notebook
from keras.layers import Conv1D,BatchNormalization,Dropout,Input,MaxPooling1D,Flatten,Dense,Input
from keras.models import Model, Sequential
def get_model(input_shape=(500,3),act='relu',loss="sparse_categorical_crossentropy",opt='adam',n_classes=350):
    model =  Sequential()
    model.add(Conv1D(100,10,input_shape=input_shape,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(5))
    model.add(BatchNormalization())
    model.add(Dropout(.2))
    model.add(Conv1D(100,10,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(2))
    model.add(BatchNormalization())
    model.add(Dropout(.2))
    model.add(Conv1D(100,10,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(2))
    model.add(BatchNormalization())
#     model.add(Conv1D(200,10,activation='relu',kernel_initializer='normal',padding='same'))
#     model.add(MaxPooling1D(2))
#     model.add(BatchNormalization())
    model.add(Dropout(.2))
    model.add(Conv1D(50,2,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(4))
    model.add(BatchNormalization())
    model.add(Dropout(.2))
    model.add(Flatten())
    model.add(Dense(n_classes,activation='sigmoid',kernel_initializer='normal'))
    model.add(Dense(n_classes,activation='softmax',kernel_initializer='normal'))
    model.compile(loss=loss,optimizer=opt,metrics=['acc'])
    return model

Using TensorFlow backend.


In [12]:
n_classes = len(np.unique(groups_train_in))
model = get_model(n_classes=n_classes)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 500, 100)          3100      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 100, 100)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 100, 100)          400       
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 100)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 100, 100)          100100    
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 50, 100)           0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 50, 100)          

In [16]:
y_train_in = y_train_in-np.min(y_train_in)
y_val_in = y_val_in-np.min(y_val_in)
y_test_in = y_test_in-np.min(y_test_in)
from keras.models import load_model
filepath = '../model_files/base_cnn_20_seconds_ppg_only.hdf5'
model = load_model(filepath)

In [17]:
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=20)
callbacks_list = [es,checkpoint]
history = model.fit(X_train_in,y_train_in,validation_data=(X_test_in,y_test_in), epochs=100, batch_size=200,
          callbacks=callbacks_list,shuffle=True)

Train on 114278 samples, validate on 31745 samples
Epoch 1/100
114278/114278 [==============================] - 38s 330us/step - loss: 3.4704 - acc: 0.2005 - val_loss: 4.0045 - val_acc: 0.1460

Epoch 00001: val_loss improved from inf to 4.00446, saving model to ../model_files/base_cnn_20_seconds_ppg_only.hdf5
Epoch 2/100
 42600/114278 [==========>...................] - ETA: 21s - loss: 3.4483 - acc: 0.2032

KeyboardInterrupt: 

In [ ]:
y_pred_in = model.predict(X_test_in)

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix

In [ ]:
np.argmax(confusion_matrix(np.argmax(y_test_in,axis=1),np.argmax(y_pred_in,axis=1)),axis=1),np.unique(groups_test_in)

In [ ]:
plt.plot(y_pred_in[40])